In [1]:
import pytesseract
import pandas as pd
from PIL import Image, ImageFilter, ImageEnhance
from pdf2image import convert_from_path
import os

TESSERACT_OCR_PATH = os.getenv('TESSERACT_OCR_PATH')
POPPLER_PATH = os.getenv('POPPLER_PATH')

def preprocess_image(image: Image):
    # Convert to grayscale
    image = image.convert('L')
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)
    # Apply sharpening
    image = image.filter(ImageFilter.SHARPEN)
    return image

def find_word(data, word, conf_threshold=10):
    filtered_data = data[data.conf >= conf_threshold]
    word_loc = filtered_data[filtered_data['text'].str.contains(word, case=False)]
    return word_loc

def filter_table_area(data:pd.DataFrame, start_word:str, stop_word:str, conf_threshold=0):
    start_word_loc = find_word(data, start_word, conf_threshold)
    stop_word_loc = find_word(data, stop_word, conf_threshold)
    filtered_data = data[(data.top >= start_word_loc.top.values[0])&(data.top < min(stop_word_loc.top.values))]
    return filtered_data

def extract_column(data, name, conf_threshold, offset=0, width=0):
    word_loc = find_word(data, name, conf_threshold)
    column = data[(data.conf > conf_threshold) & (data.left >= word_loc.left.values[0]+offset) & (data.left < word_loc.left.values[0]+width)& (data.top >= word_loc.top.values[0])]
    return column

def join_words_in_line(data, line_num):
    desc = ''
    for text in data[data.line_num == line_num].text:
        if desc == '':
            desc = text
        else:
            desc = " ".join([desc, text])
    return desc

# images = convert_from_path("test_data\S247247 RB Collection.pdf", dpi=300, poppler_path=POPPLER_PATH)
# images = convert_from_path("test_data\S246907 RB COLLECTION.pdf", dpi=300, poppler_path=POPPLER_PATH)
# images = convert_from_path("test_data\S246218 RB COLLECTION.pdf", dpi=300, poppler_path=POPPLER_PATH)
images = convert_from_path("test_data\S248659 RB COLLECTION.pdf", dpi=300, poppler_path=POPPLER_PATH)

all_processed_image_data = []

for image in images:
    # image_processed = preprocess_image(image)
    image_processed = image
    processed_image_data = pytesseract.image_to_data(image_processed, output_type=pytesseract.Output.DATAFRAME)
    all_processed_image_data.append(processed_image_data)


In [2]:
column_list = ['line no', 'qty', 'item', 'description', 'unit price', 'disc price', 'ext price']

extracted_table = pd.DataFrame(columns=column_list)

In [3]:
extracted_table

,line no,qty,item,description,unit price,disc price,ext price


In [4]:
def data_validation(data, regex):
    validated_data = data[data['text'].str.contains(regex)]
    return validated_data

In [5]:
# for each page, extract invoiced item data
for i in range(len(all_processed_image_data)):
    
    # filter table area
    if i == len(all_processed_image_data)-1:
        table_data = filter_table_area(all_processed_image_data[i], 'qty', 'total')
        print('total')
    else:
        table_data = filter_table_area(all_processed_image_data[i], 'qty', 'continued')
        print('continued')

    # extract columns
    qty = extract_column(table_data, 'qty', 0, 0, 90)
    item = extract_column(table_data, 'item', 10, -180, 150)
    description = extract_column(table_data, 'description', 40, -17*30, 20*30)
    unit_price = extract_column(table_data, 'unit', 10, 0, 8*30)
    disc_price = extract_column(table_data, 'disc', 10, 0, 8*30)
    ext_price = extract_column(table_data, 'ext', 0, 0, 8*30)

    line_list = qty.line_num.unique()

    temp_table = pd.DataFrame(columns=column_list)

    if len(line_list) > 1:
        temp_table['line no'] = line_list[1:]
        temp_table['qty'] = data_validation(qty, r'^\d+$').text.values
        temp_table['item'] = item.text.values[1:]
        temp_table['unit price'] = data_validation(unit_price, r'^\d+\.\d{2}$').text.values
        temp_table['disc price'] = data_validation(disc_price, r'^\d+\.\d{2}$').text.values
        temp_table['ext price'] = data_validation(ext_price, r'^\d+\.\d{2}$').text.values

        for i in range(len(temp_table)):
            temp_table.loc[i, 'description'] = join_words_in_line(description, temp_table['line no'][i])
            temp_table.loc[i, 'description'] += " #"
            if join_words_in_line(description, temp_table['line no'][i]+1):
                temp_table.loc[i, 'description'] += join_words_in_line(description, temp_table['line no'][i]+1)
            else:
                temp_table.loc[i, 'description'] += join_words_in_line(description, 1)

    extracted_table = pd.concat([extracted_table, temp_table], ignore_index=True)

extracted_table

continued
total


,line no,qty,item,description,unit price,disc price,ext price
0,9,4,PA-TERRIAN,#NATURAL-4,93.50,64.99,259.96
1,11,4,DEENA,100% UNPROCESSED BRAZILIAN HD LACE #NATURAL-2 ...,95.00,59.99,239.96
2,13,12,DEEP182022,"DEEP WAVE 18""20""22"" #1B-4 2-4 P1B30-4",14.95,12.95,155.40
3,15,4,JC18/20/22,JERRY CURL 18-20-22 #1-4,14.95,12.95,51.80
4,17,4,LORRELL,100% UNPROCESSED BRAZILIAN HD LACE #NATURAL-2 ...,95.00,59.99,239.96
5,19,6,SUP-BULK18,"SUPER BULK 18"" #1B-6",8.00,7.00,42.00
6,21,8,SUP-BULK22,"SUPER BULK 22"" #1-4 1B-4",9.00,8.00,64.00
7,23,4,U-ASIA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,81.95,74.99,299.96
8,25,2,U-ELSA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,86.95,74.99,149.98
9,27,4,U-VESA,100% UNPROCESSED BRAZILIAN VIRGIN REMI #NATURA...,68.50,66.50,266.00


In [9]:
extracted_table.drop('line no', axis=1).to_excel('RB_invoice.xlsx', index=False)